In [5]:
import time
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
# from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from pandas.plotting import parallel_coordinates
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler # 표준화(평균을 기준)
from sklearn.preprocessing import MinMaxScaler # 정규화(정해진 규칙 기준, mix-max기준)
from sklearn.datasets import load_iris
# cross_val_score: 다양한 샘플링으 ㄹ해서 여러번 테스트 --> 평균치를 구해본다.!
from sklearn.metrics import *
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import LabelEncoder


%matplotlib inline

# 주요 라이브러리 import

from matplotlib import rc, font_manager
import matplotlib.font_manager as fm


# 디렉토리 및 파일 이름에 맞추어 변경
font_location = '../NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
rc('font', family=font_name)
import scipy.io as spio

from pandas.plotting import parallel_coordinates

In [25]:
df_raw = pd.read_csv('1999년 전국민 일반csv.csv')
df_raw

,Unnamed: 0,id,area,area1,sex,age,q1,q2,q3,q4,...,q15_22,q16,q17,q18,q19,q20,occu,religion,income,educ
0,1,1224.0,32.0,1.0,2.0,39.0,4.0,93.0,62.0,2.0,...,1.0,5.0,88.0,1.0,1.0,1.0,54.0,4.0,9999.0,12.0
1,2,1014.0,11.0,2.0,1.0,46.0,6.0,999.0,63.0,2.0,...,1.0,40.0,0.0,2.0,1.0,9.0,45.0,3.0,250.0,9.0
2,3,267.0,34.0,1.0,2.0,29.0,4.0,241.0,181.0,1.0,...,1.0,40.0,3.0,1.0,2.0,9.0,52.0,3.0,200.0,16.0
3,4,10.0,1.0,2.0,1.0,30.0,4.0,0.0,85.0,2.0,...,1.0,25.0,10.0,1.0,9.0,9.0,34.0,4.0,130.0,12.0
4,5,38.0,2.0,2.0,1.0,48.0,5.0,64.0,83.0,2.0,...,3.0,20.0,10.0,2.0,2.0,2.0,21.0,6.0,250.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,1368,264.0,35.0,1.0,1.0,30.0,4.0,180.0,16.0,2.0,...,2.0,66.0,5.0,1.0,2.0,2.0,42.0,6.0,125.0,12.0
1368,1369,13.0,11.0,2.0,2.0,23.0,5.0,65.0,999.0,2.0,...,1.0,20.0,5.0,2.0,3.0,3.0,21.0,6.0,9999.0,14.0
1369,1370,1494.0,1.0,2.0,1.0,60.0,3.0,362.0,63.0,2.0,...,3.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,16.0
1370,1371,1253.0,2.0,2.0,1.0,35.0,6.0,999.0,999.0,2.0,...,9.0,0.0,0.0,4.0,1.0,1.0,31.0,1.0,120.0,12.0


In [26]:
## 국내여행자 372명
df_raw['q4'].value_counts()

2.0    998
1.0    372
0.0      2
Name: q4, dtype: int64

In [27]:
## 국내 여행만 뽑아서 인덱스 리셋
df_korea = df_raw[df_raw['q4']==1.0].reset_index()
df_korea

,index,Unnamed: 0,id,area,area1,sex,age,q1,q2,q3,...,q15_22,q16,q17,q18,q19,q20,occu,religion,income,educ
0,2,3,267.0,34.0,1.0,2.0,29.0,4.0,241.0,181.0,...,1.0,40.0,3.0,1.0,2.0,9.0,52.0,3.0,200.0,16.0
1,9,10,1079.0,1.0,2.0,1.0,30.0,6.0,140.0,81.0,...,9.0,12.0,6.0,2.0,1.0,3.0,31.0,6.0,250.0,16.0
2,14,15,1182.0,4.0,2.0,1.0,31.0,8.0,20.0,888.0,...,3.0,8.0,0.0,1.0,1.0,1.0,21.0,6.0,300.0,14.0
3,15,16,396.0,4.0,2.0,2.0,48.0,5.0,83.0,63.0,...,2.0,8.0,12.0,2.0,2.0,3.0,52.0,1.0,250.0,16.0
4,17,18,611.0,4.0,2.0,2.0,45.0,7.0,81.0,62.0,...,1.0,6.0,3.0,2.0,2.0,1.0,31.0,1.0,130.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,1345,1346,60.0,1.0,2.0,1.0,37.0,4.0,83.0,114.0,...,1.0,66.0,10.0,2.0,2.0,2.0,5.0,6.0,400.0,18.0
368,1346,1347,1541.0,26.0,2.0,1.0,31.0,4.0,348.0,109.0,...,1.0,30.0,4.0,2.0,1.0,2.0,53.0,4.0,200.0,16.0
369,1347,1348,1525.0,11.0,2.0,1.0,40.0,4.0,84.0,251.0,...,3.0,30.0,8.0,2.0,1.0,9.0,21.0,3.0,150.0,0.0
370,1354,1355,599.0,1.0,2.0,1.0,44.0,4.0,63.0,888.0,...,1.0,24.0,2.0,2.0,2.0,2.0,21.0,6.0,200.0,16.0


In [28]:
list(df_korea.columns)

['index',
 'Unnamed: 0',
 'id',
 'area',
 'area1',
 'sex',
 'age',
 'q1',
 'q2',
 'q3',
 'q4',
 'q4_1',
 'q4_2',
 'q5',
 'q5_1',
 'q5_2',
 'q5_3',
 'q5_4',
 'q6',
 'q7',
 'q7_1',
 'q7_2',
 'q8',
 'q9',
 'q10',
 'q11',
 'q12',
 'q13_11',
 'q13_12',
 'q13_13',
 'q13_21',
 'q13_22',
 'q13_23',
 'q13_31',
 'q13_32',
 'q13_33',
 'q13_41',
 'q13_42',
 'q13_43',
 'q13_51',
 'q13_52',
 'q13_53',
 'q13_61',
 'q13_62',
 'q13_63',
 'q13_71',
 'q13_72',
 'q13_73',
 'q13_81',
 'q13_82',
 'q13_83',
 'q13_91',
 'q13_92',
 'q13_93',
 'q13_101',
 'q13_102',
 'q13_103',
 'q13_111',
 'q13_112',
 'q13_113',
 'q13_121',
 'q13_122',
 'q13_123',
 'q13_131',
 'q13_132',
 'q13_133',
 'q13_141',
 'q13_142',
 'q13_143',
 'q13_151',
 'q13_152',
 'q13_153',
 'q13_161',
 'q13_162',
 'q13_163',
 'q13_171',
 'q13_172',
 'q13_173',
 'q14',
 'q14_1',
 'q15_1',
 'q15_2',
 'q15_3',
 'q15_4',
 'q15_5',
 'q15_6',
 'q15_7',
 'q15_8',
 'q15_9',
 'q15_10',
 'q15_11',
 'q15_12',
 'q15_13',
 'q15_14',
 'q15_15',
 'q15_16',
 'q1

In [21]:
## 불필요 컬럼 드랍
df_korea.drop(columns=['q1', 'q2', 'q3', 'q4', 'q5', 'q10', 'q16', 'q17', 'q18', 'occu', 'religion', 'income', 'educ'], inplace=True)
df_korea

,index,Unnamed: 0,id,area,area1,sex,age,q4_1,q4_2,q5_1,...,q15_15,q15_16,q15_17,q15_18,q15_19,q15_20,q15_21,q15_22,q19,q20
0,2,3,267.0,34.0,1.0,2.0,29.0,2.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,9.0
1,9,10,1079.0,1.0,2.0,1.0,30.0,4.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,9.0,9.0,9.0,1.0,3.0
2,14,15,1182.0,4.0,2.0,1.0,31.0,1.0,2.0,0.0,...,1.0,1.0,1.0,1.0,3.0,1.0,3.0,3.0,1.0,1.0
3,15,16,396.0,4.0,2.0,2.0,48.0,1.0,2.0,0.0,...,1.0,1.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0,3.0
4,17,18,611.0,4.0,2.0,2.0,45.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,1345,1346,60.0,1.0,2.0,1.0,37.0,9.0,2.0,0.0,...,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,2.0,2.0
368,1346,1347,1541.0,26.0,2.0,1.0,31.0,3.0,2.0,0.0,...,2.0,2.0,2.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0
369,1347,1348,1525.0,11.0,2.0,1.0,40.0,2.0,2.0,0.0,...,1.0,1.0,3.0,1.0,1.0,1.0,1.0,3.0,1.0,9.0
370,1354,1355,599.0,1.0,2.0,1.0,44.0,1.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0


In [22]:
df_korea.columns

Index(['index', 'Unnamed: 0', 'id', 'area', 'area1', 'sex', 'age', 'q4_1',
       'q4_2', 'q5_1', 'q5_2', 'q5_3', 'q5_4', 'q6', 'q7', 'q7_1', 'q7_2',
       'q8', 'q9', 'q11', 'q12', 'q13_11', 'q13_12', 'q13_13', 'q13_21',
       'q13_22', 'q13_23', 'q13_31', 'q13_32', 'q13_33', 'q13_41', 'q13_42',
       'q13_43', 'q13_51', 'q13_52', 'q13_53', 'q13_61', 'q13_62', 'q13_63',
       'q13_71', 'q13_72', 'q13_73', 'q13_81', 'q13_82', 'q13_83', 'q13_91',
       'q13_92', 'q13_93', 'q13_101', 'q13_102', 'q13_103', 'q13_111',
       'q13_112', 'q13_113', 'q13_121', 'q13_122', 'q13_123', 'q13_131',
       'q13_132', 'q13_133', 'q13_141', 'q13_142', 'q13_143', 'q13_151',
       'q13_152', 'q13_153', 'q13_161', 'q13_162', 'q13_163', 'q13_171',
       'q13_172', 'q13_173', 'q14', 'q14_1', 'q15_1', 'q15_2', 'q15_3',
       'q15_4', 'q15_5', 'q15_6', 'q15_7', 'q15_8', 'q15_9', 'q15_10',
       'q15_11', 'q15_12', 'q15_13', 'q15_14', 'q15_15', 'q15_16', 'q15_17',
       'q15_18', 'q15_19', 'q15_20',